In [2]:
from cgitb import text
import spacy
import os
import csv
import pandas as pd
import numpy as np
from tqdm import tqdm
from spacy.tokens import DocBin
from multiprocessing import Process, freeze_support, set_start_method

In [26]:
def load_word_vectors(model_name, word_vectors):
    import spacy
    import subprocess
    import sys
    subprocess.run([sys.executable,
                    "-m",
                    "spacy",
                    "init",
                    "vectors",
                    "en",
                    word_vectors,
                    model_name,
                        ]
                    )
    print (f"New spaCy model created with word vectors. File: {model_name}")

In [103]:
spacy.prefer_gpu()
nlp = spacy.load("./models/model-best")

dataframe = pd.read_csv('./dataset/Train_Tagged_Titles.tsv', sep='\t', on_bad_lines='skip', quoting=csv.QUOTE_NONE, encoding='utf8')
dataframe2 = pd.read_csv('./dataset/Listing_Titles.tsv', sep='\t', on_bad_lines='skip', quoting=csv.QUOTE_NONE, encoding='utf8')

In [104]:
listing_titles = [(dataframe2["Title"][i], i) for i in range(0, len(dataframe2))][0:5000:10]


In [105]:
tagged_titles = pd.read_csv('./dataset/Train_Tagged_Titles.tsv', sep='\t', on_bad_lines='skip', quoting=csv.QUOTE_NONE, encoding='utf8')

tokens = tagged_titles.groupby('Record Number')['Token'].apply(list).to_dict()
tags = tagged_titles.groupby('Record Number')['Tag'].apply(list).to_dict()

raw_tokenized_data = [[(tokens[i][tiddie], tags[i][tiddie]) for tiddie in range(0, len(tags[i]))] for i in range(1, len(tags) + 1)]

for i in range(0, len(raw_tokenized_data)):
    for j in reversed(range(1, len(raw_tokenized_data[i]))):
        if (raw_tokenized_data[i][j][1] != raw_tokenized_data[i][j][1]): #python nan moment
            raw_tokenized_data[i][j - 1] = (raw_tokenized_data[i][j - 1][0] + " " + raw_tokenized_data[i][j][0], raw_tokenized_data[i][j - 1][1])
trimmed_tokenized_data = [[i for i in item if not i[1] != i[1]] for item in raw_tokenized_data]

trimmed_tokenized_data
df2 = pd.DataFrame(columns= ['Record Number', 'Token', 'expected'])
for i in range(0, len(trimmed_tokenized_data)):
    for j in range(0, len(trimmed_tokenized_data[i])) :
        if(i % 10 == 0):
            df2.loc[len(df2)] = [i + 1, trimmed_tokenized_data[i][j][0], trimmed_tokenized_data[i][j][1]]






In [107]:
df = pd.DataFrame(columns= ['Record Number', 'Token', 'predicted'])
result = []

with tqdm(total = 500) as pbar:
    buffer = ""
    for doc, i in nlp.pipe(listing_titles, as_tuples=True, batch_size=10000):
        for ent in doc.ents:
            # if(ent.label_ != "No Tag" and ent.label_ != "Obscure"):
                result.append(str(i + 1) + '\t' + ent.label_ + '\t' + ent.text)
                df.loc[len(df)] = [i + 1, ent.text, ent.label_]
        pbar.update(1)

df = pd.merge(df, df2, how = 'left')

frame = pd.DataFrame(df)
frame.to_csv('failure_analysis.csv')
        
output = open("output/output.tsv", "w", encoding="utf-8")  # write mode
output.write('\n'.join(result))
output.close()

100%|██████████| 500/500 [00:06<00:00, 73.74it/s] 
